## implementação dos algoritmos de recorte

In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import xml.etree.ElementTree as ET
import math


class Visualizador:
    def __init__(self, root):
        self.root = root
        self.root.title("Visualizador de Objetos 2D")
    
        # Configurar o menu
        menu = tk.Menu(root)
        root.config(menu=menu)
        file_menu = tk.Menu(menu, tearoff=False)
        menu.add_cascade(label="Arquivo", menu=file_menu)
        file_menu.add_command(label="Abrir", command=self.abrir_arquivo)
        file_menu.add_command(label="Gerar XML de Saída", command=self.gerar_xml_saida)

        # Adiciona o menu "Sobre"
        sobre_menu = tk.Menu(menu, tearoff=False)
        menu.add_cascade(label="Sobre", menu=sobre_menu)
        sobre_menu.add_command(label="Controles do Teclado", command=self.mostrar_controles_do_teclado)
        sobre_menu.add_command(label="DisplayList", command=self.abrir_display_list)
    
        # Frame principal para conter canvas e minimapa
        frame_principal = tk.Frame(root)
        frame_principal.pack(side="top", fill="both", expand=True)
    
        # canvas da Viewport principal
        self.canvas = tk.Canvas(frame_principal, width=800, height=600, bg="white")
        self.canvas.pack(side="left", fill="both", expand=True)
    
        # Painel da direita com minimapa e seletor de algoritmos
        right_panel = tk.Frame(frame_principal)
        right_panel.pack(side="right", fill="y", padx=10, pady=10)

        # Espaçador superior para descer os componentes
        tk.Label(right_panel, text="").pack(pady=(200, 0))

        # Canvas do minimapa
        self.minimap = tk.Canvas(right_panel, width=150, height=120, bg="lightgrey")
        self.minimap.pack(padx=0, pady=(0, 10))

        # Seletor de algoritmo de clipping de linhas
        grp_linhas = tk.LabelFrame(right_panel, text="Recorte (Linhas)", padx=5, pady=5)
        grp_linhas.pack(fill="x", padx=0, pady=(0, 10))

        self.algoritmo_linha = tk.StringVar(value="Cohen-Sutherland")
        tk.Radiobutton(grp_linhas, text="Cohen", variable=self.algoritmo_linha,
                       value="Cohen-Sutherland", 
                       command=self._processar_e_desenhar).pack(anchor="w")
        tk.Radiobutton(grp_linhas, text="Liang", variable=self.algoritmo_linha,
                       value="Liang-Barsky", 
                       command=self._processar_e_desenhar).pack(anchor="w")

        # Informação sobre algoritmo de polígonos
        grp_poligonos = tk.LabelFrame(right_panel, text="Recorte (Polígonos)", padx=5, pady=5)
        grp_poligonos.pack(fill="x", padx=0, pady=0)
        tk.Label(grp_poligonos, text="→ Weiler-Atherton", 
                 font=("Arial", 9)).pack(anchor="w")

        # Frame para os botões de navegação na parte inferior
        frame_botoes = tk.Frame(root)
        frame_botoes.pack(side="bottom", anchor="w", padx=10, pady=10)

        # Botões de navegação
        btn_up = tk.Button(frame_botoes, text="↑", width=3, command=lambda: self._mover_e_recarregar(0, 1))
        btn_down = tk.Button(frame_botoes, text="↓", width=3, command=lambda: self._mover_e_recarregar(0, -1))
        btn_left = tk.Button(frame_botoes, text="←", width=3, command=lambda: self._mover_e_recarregar(-1, 0))
        btn_right = tk.Button(frame_botoes, text="→", width=3, command=lambda: self._mover_e_recarregar(1, 0))

        # Posiciona os botões em uma grade
        btn_up.grid(row=0, column=1, padx=3, pady=3)
        btn_left.grid(row=1, column=0, padx=3, pady=3)
        btn_right.grid(row=1, column=2, padx=3, pady=3)
        btn_down.grid(row=2, column=1, padx=3, pady=3)

        # Botões de Zoom
        btn_zoom_in = tk.Button(frame_botoes, text="+", width=3, command=lambda: self._zoom(1.1)) 
        btn_zoom_out = tk.Button(frame_botoes, text="-", width=3, command=lambda: self._zoom(0.9))
        btn_zoom_in.grid(row=1, column=3, padx=(15, 3))
        btn_zoom_out.grid(row=1, column=4, padx=3)

        # Botões de Rotação
        btn_rot_left = tk.Button(frame_botoes, text="⟲", width=3, command=lambda: self._rotacionar(-2.0))
        btn_rot_right = tk.Button(frame_botoes, text="⟳", width=3, command=lambda: self._rotacionar(2.0))
        btn_rot_left.grid(row=1, column=5, padx=(15, 3))
        btn_rot_right.grid(row=1, column=6, padx=3)

        # Lista de objetos lidos do XML
        self.objects = []
        self.window = {'wmin':(0.0,0.0), 'wmax':(1.0,1.0)}
        self.viewport = {'vpmin':(0.0,0.0), 'vpmax':(800.0,600.0)}
        
        # Ângulo de rotação da window em graus
        self.rotation_angle = 0.0
        
        self.world_bounds = {'xmin': 0.0, 'xmax': 1.0, 'ymin': 0.0, 'ymax': 1.0}
        self.caminho_arquivo = None

        # Margem para a região de clipping
        self.clipping_margin = 30

        # Referência para janela DisplayList
        self.displaylist_window = None
        self.displaylist_tree = None

        # Bind das teclas
        root.bind("<Left>", lambda e: self._mover_e_recarregar(-1,0))
        root.bind("<Right>", lambda e: self._mover_e_recarregar(1,0))
        root.bind("<Up>", lambda e: self._mover_e_recarregar(0,1))
        root.bind("<Down>", lambda e: self._mover_e_recarregar(0,-1))
        root.bind("<plus>", lambda e: self._zoom(0.9))
        root.bind("<equal>", lambda e: self._zoom(0.9))
        root.bind("<KP_Add>", lambda e: self._zoom(0.9))
        root.bind("<minus>", lambda e: self._zoom(1.1))
        root.bind("<KP_Subtract>", lambda e: self._zoom(1.1))
        root.bind("<comma>", lambda e: self._rotacionar(-2.0))
        root.bind("<period>", lambda e: self._rotacionar(2.0))
        
    # função para rotacionar a window
    def _rotacionar(self, angulo):
        self.rotation_angle += angulo
        self._processar_e_desenhar()

    def abrir_arquivo(self):
        caminho = filedialog.askopenfilename(filetypes=[("XML files","*.xml"),("All files","*.*")])
        if not caminho:
            return
        try:
            self.caminho_arquivo = caminho
            self.carregar_arquivo(caminho)
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao carregar arquivo:\n{e}")
            
    # ler os dados do arquivo aberto
    def carregar_arquivo(self, caminho):
        tree = ET.parse(caminho)
        root = tree.getroot()

        # Parse viewport
        vp = root.find('viewport')
        if vp is not None:
            vpmin = vp.find('vpmin')
            vpmax = vp.find('vpmax')
            if vpmin is not None and vpmax is not None:
                try:
                    self.viewport['vpmin'] = (float(vpmin.get('x', '0')), float(vpmin.get('y', '0')))
                    self.viewport['vpmax'] = (float(vpmax.get('x', '800')), float(vpmax.get('y', '600')))
                except ValueError:
                    pass

        # Ajusta canvas ao tamanho do viewport lido
        vpxmin, vpymin = self.viewport['vpmin']
        vpxmax, vpymax = self.viewport['vpmax']
        largura = max(200, int(abs(vpxmax - vpxmin)))
        altura = max(200, int(abs(vpymax - vpymin)))
        self.canvas.config(width=largura, height=altura)

        # Parse window
        w = root.find('window')
        if w is not None:
            wmin = w.find('wmin')
            wmax = w.find('wmax')
            if wmin is not None and wmax is not None:
                try:
                    self.window['wmin'] = (float(wmin.get('x','0')), float(wmin.get('y','0')))
                    self.window['wmax'] = (float(wmax.get('x','1')), float(wmax.get('y','1')))
                except ValueError:
                    pass

        # Parse objetos
        self.objects = []
        for child in root:
            tag = child.tag.lower() if child.tag else ''
            # ignorar objetos marcados coords="viewport
            if child.get('coords') == 'viewport':
                continue
            if tag == 'viewport' or tag == 'window':
                continue
            color = child.get('cor', 'black')
            
            if tag == 'ponto':
                try:
                    x = float(child.get('x','0'))
                    y = float(child.get('y','0'))
                    self.objects.append({
                        'type':'ponto', 
                        'coords_mundo':[(x,y)],
                        'coords_ppc': [],
                        'coords_ppc_clipped': [],
                        'coords_viewport': [],
                        'color': color,
                        'visivel': True
                    })
                except ValueError:
                    continue
            elif tag == 'reta':
                pts = []
                for p in child.findall('ponto'):
                    try:
                        x = float(p.get('x','0'))
                        y = float(p.get('y','0'))
                        pts.append((x, y))
                    except ValueError:
                        continue
                if len(pts) >= 2:
                    self.objects.append({
                        'type':'reta', 
                        'coords_mundo':pts[:2],
                        'coords_ppc': [],
                        'coords_ppc_clipped': [],
                        'coords_viewport': [],
                        'color': color,
                        'visivel': True
                    })
            elif tag == 'poligono':
                pts = []
                for p in child.findall('ponto'):
                    try:
                        x = float(p.get('x','0'))
                        y = float(p.get('y','0'))
                        pts.append((x, y))
                    except ValueError:
                        continue
                if len(pts) >= 3:
                    self.objects.append({
                        'type':'poligono', 
                        'coords_mundo':pts,
                        'coords_ppc': [],
                        'coords_ppc_clipped': [],
                        'coords_viewport': [],
                        'color': color,
                        'visivel': True
                    })

        # Calcula os limites do mundo
        self.calcular_limites_mundo()
        
        self.window['wmin'] = (self.world_bounds['xmin'], self.world_bounds['ymin'])
        self.window['wmax'] = (self.world_bounds['xmax'], self.world_bounds['ymax'])
        self.rotation_angle = 0.0

        # Processa e desenha
        self._processar_e_desenhar()
     
    # calcula os limites do mundo baseado nos objetos carregados   
    def calcular_limites_mundo(self):
        if not self.objects:
            # usa window como base se não houver objetos
            wxmin, wymin = self.window['wmin']
            wxmax, wymax = self.window['wmax']
            self.world_bounds = {'xmin': wxmin, 'xmax': wxmax, 'ymin': wymin, 'ymax': wymax}
            return
        
        xs = []
        ys = []
        for obj in self.objects:
            for x, y in obj['coords_mundo']:
                xs.append(x)
                ys.append(y)
                
        if xs and ys:
            xmin, xmax = min(xs), max(xs)
            ymin, ymax = min(ys), max(ys)
            # margem relativa 30% do tamanho atual da window
            wwidth = self.window['wmax'][0] - self.window['wmin'][0]
            wheight = self.window['wmax'][1] - self.window['wmin'][1]
            margin_x = abs(wwidth) * 0.3 if wwidth != 0 else 1.0
            margin_y = abs(wheight) * 0.3 if wheight != 0 else 1.0
            self.world_bounds = {
                'xmin': xmin - margin_x,
                'xmax': xmax + margin_x,
                'ymin': ymin - margin_y,
                'ymax': ymax + margin_y
            }

    # Transforma todos os objetos do sistema de coordenadas do mundo para o Sistema de Coordenadas do Plano de Projeção (PPC)
    def transformar_para_ppc(self):
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        
        window_center_x = (wminx + wmaxx) / 2
        window_center_y = (wminy + wmaxy) / 2
        
        # Ângulo de rotação em radianos
        angle_rad = math.radians(-self.rotation_angle)
        cos_a = math.cos(angle_rad)
        sin_a = math.sin(angle_rad)
        
        for obj in self.objects:
            obj['coords_ppc'] = []
            for x, y in obj['coords_mundo']:
                # Translada para o centro da window
                tx = x - window_center_x
                ty = y - window_center_y
                
                # Rotaciona
                rx = tx * cos_a - ty * sin_a
                ry = tx * sin_a + ty * cos_a
                
                # Translada de volta
                px = rx + window_center_x
                py = ry + window_center_y
                
                obj['coords_ppc'].append((px, py))
    
    # Calcula o código de região para Cohen-Sutherland
    def calcular_codigo_regiao(self, x, y):
        wxmin, wymin = self.window['wmin']
        wxmax, wymax = self.window['wmax']
        
        codigo = 0
        if y > wymax:
            codigo |= 8  # Acima(1000)
        elif y < wymin:
            codigo |= 4  # Abaixo(0100)
        if x > wxmax:
            codigo |= 2  # Direita(0010)
        elif x < wxmin:
            codigo |= 1  # Esquerda(0001)
        return codigo
    
    # Clipping de linha usando Cohen-Sutherland
    def cohen_sutherland_clip(self, x1, y1, x2, y2):
        wxmin, wymin = self.window['wmin']
        wxmax, wymax = self.window['wmax']
        
        c1 = self.calcular_codigo_regiao(x1, y1)
        c2 = self.calcular_codigo_regiao(x2, y2)
        
        aceito = False
        
        while True:
            # Aceitação trivial
            if c1 == 0 and c2 == 0:
                aceito = True
                break
            # Rejeição trivial
            elif (c1 & c2) != 0:
                break
            else:
                # Escolhe um ponto fora
                c_fora = c1 if c1 != 0 else c2
                
                # Calcula intersecção
                if c_fora & 8:  # acima
                    x = x1 + (x2 - x1) * (wymax - y1) / (y2 - y1) if (y2 - y1) != 0 else x1
                    y = wymax
                elif c_fora & 4:  # abaixo
                    x = x1 + (x2 - x1) * (wymin - y1) / (y2 - y1) if (y2 - y1) != 0 else x1
                    y = wymin
                elif c_fora & 2:  # direita
                    y = y1 + (y2 - y1) * (wxmax - x1) / (x2 - x1) if (x2 - x1) != 0 else y1
                    x = wxmax
                elif c_fora & 1:  # esquerda
                    y = y1 + (y2 - y1) * (wxmin - x1) / (x2 - x1) if (x2 - x1) != 0 else y1
                    x = wxmin
                
                # Atualiza o ponto que estava fora
                if c_fora == c1:
                    x1, y1 = x, y
                    c1 = self.calcular_codigo_regiao(x1, y1)
                else:
                    x2, y2 = x, y
                    c2 = self.calcular_codigo_regiao(x2, y2)
        
        if aceito:
            return True, (x1, y1), (x2, y2)
        else:
            return False, None, None
    
    # Clipping de linha usando Liang-Barsky
    def liang_barsky_clip(self, x1, y1, x2, y2):
        wxmin, wymin = self.window['wmin']
        wxmax, wymax = self.window['wmax']
        
        dx = x2 - x1
        dy = y2 - y1
        
        u1 = 0.0
        u2 = 1.0
        
        # p e q para as 4 bordas
        p = [-dx, dx, -dy, dy]
        q = [x1 - wxmin, wxmax - x1, y1 - wymin, wymax - y1]
        
        for i in range(4):
            if p[i] == 0:
                # Linha paralela à borda
                if q[i] < 0:
                    return False, None, None
            else:
                r = q[i] / p[i]
                if p[i] < 0:
                    # Entrando
                    u1 = max(u1, r)
                else:
                    # Saindo
                    u2 = min(u2, r)
        
        if u1 > u2:
            return False, None, None
        
        # Calcula novos pontos
        x1_new = x1 + u1 * dx
        y1_new = y1 + u1 * dy
        x2_new = x1 + u2 * dx
        y2_new = y1 + u2 * dy
        
        return True, (x1_new, y1_new), (x2_new, y2_new)
    
    def weiler_atherton_clip(self, vertices):
        # Recorte interseção entre P e W
        wxmin, wymin = self.window['wmin']
        wxmax, wymax = self.window['wmax']

        def inside_window(p):
            x, y = p
            return (wxmin - 1e-12) <= x <= (wxmax + 1e-12) and (wymin - 1e-12) <= y <= (wymax + 1e-12)

        # Monta polígono da janela
        clip_poly = [(wxmin, wymin), (wxmax, wymin), (wxmax, wymax), (wxmin, wymax)]

        # Utilitários geométricos
        def seg_intersection(a, b, c, d):
            (x1, y1), (x2, y2) = a, b
            (x3, y3), (x4, y4) = c, d
            den = (x1 - x2)*(y3 - y4) - (y1 - y2)*(x3 - x4)
            if abs(den) < 1e-12:
                return (False, None, None, None)
            t = ((x1 - x3)*(y3 - y4) - (y1 - y3)*(x3 - x4)) / den
            u = ((x1 - x3)*(y1 - y2) - (y1 - y3)*(x1 - x2)) / den
            if -1e-12 <= t <= 1 + 1e-12 and -1e-12 <= u <= 1 + 1e-12:
                px = x1 + t*(x2 - x1)
                py = y1 + t*(y2 - y1)
                return (True, (px, py), max(0.0, min(1.0, t)), max(0.0, min(1.0, u)))
            return (False, None, None, None)

        # Nó com metadados
        def make_node(p, is_inter=False):
            return {'pt': p, 'inter': is_inter, 'entry': None, 'visited': False, 't': None, 'twin': None}

        # Cria listas de nós (cíclicas) para polígono e janela
        subj_nodes = [make_node(p) for p in vertices]
        clip_nodes = [make_node(p) for p in clip_poly]

        # Inserir interseções em ambas as listas, mantendo ordem por parâmetro t
        def insert_sorted(nodes, idx, node):
            # insere após idx avançando para manter ordem por t ao longo da aresta idx->idx+1
            i_next = (idx + 1) % len(nodes)
            # encontrar posição entre idx e i_next
            pos = (idx + 1) % (len(nodes) + 1)
            # varre para frente contando apenas interseções na mesma aresta
            k = (idx + 1) % len(nodes)
            count = 0
            while count < len(nodes) and nodes[k]['inter'] and nodes[k].get('edge_start') == idx:
                if node['t'] < nodes[k]['t']:
                    break
                idx = k
                k = (k + 1) % len(nodes)
                count += 1
            nodes.insert((idx + 1), node)

            # Marca edge_start para interseções ao inserir
        def add_intersections():
            # Coleta todas interseções
            for i in range(len(subj_nodes)):
                a = subj_nodes[i]['pt']
                b = subj_nodes[(i + 1) % len(subj_nodes)]['pt']
                for j in range(len(clip_nodes)):
                    c = clip_nodes[j]['pt']
                    d = clip_nodes[(j + 1) % len(clip_nodes)]['pt']
                    ok, p, ta, tc = seg_intersection(a, b, c, d)
                    if not ok:
                        continue
                    # cria nós gêmeos
                    n_sub = make_node(p, True); n_clip = make_node(p, True)
                    n_sub['t'] = ta; n_sub['edge_start'] = i
                    n_clip['t'] = tc; n_clip['edge_start'] = j
                    n_sub['twin'] = n_clip; n_clip['twin'] = n_sub
                    insert_at_i = i
                    insert_at_j = j
                    # insere
                    insert_sorted(subj_nodes, insert_at_i, n_sub)
                    insert_sorted(clip_nodes, insert_at_j, n_clip)

        add_intersections()

        # Classificar interseções como entrantes/saíntes no polígono
        def classify_entries():
            # Para cada interseção na lista do polígono, verifica inside antes/depois
            for idx, node in enumerate(subj_nodes):
                if not node['inter']:
                    continue
                prev_pt = subj_nodes[(idx - 1) % len(subj_nodes)]['pt']
                next_pt = subj_nodes[(idx + 1) % len(subj_nodes)]['pt']
                was_inside = inside_window(prev_pt)
                will_inside = inside_window(next_pt)
                node['entry'] = (not was_inside) and will_inside

            # Propaga rótulos para gêmeas
            for node in subj_nodes:
                if node['inter']:
                    node['twin']['entry'] = node['entry']

        classify_entries()

        # Caso sem interseções
        has_intersections = any(n['inter'] for n in subj_nodes)
        if not has_intersections:
            if all(inside_window(p) for p in vertices):
                return [vertices[:]]
            else:
                return []

        # Travessia
        result = []

        def advance(nodes, start_idx):
            # avança ao próximo nó (cíclico)
            return (start_idx + 1) % len(nodes)

        def follow_path(start_node):
            poly = []
            node = start_node
            in_subj = True
            current_list = subj_nodes
            idx = current_list.index(node)
            while True:
                node['visited'] = True if node['inter'] else node.get('visited', False)
                poly.append(node['pt'])
                if node['inter']:
                    # muda de lista
                    twin = node['twin']
                    current_list = clip_nodes if in_subj else subj_nodes
                    idx = current_list.index(twin)
                    node = current_list[idx]
                    in_subj = not in_subj
                # avança
                idx = advance(current_list, idx)
                node = current_list[idx]
                # termina quando retorna ao início
                if node is start_node:
                    poly.append(node['pt'])
                    break
            # remove ponto duplicado final
            if len(poly) > 1 and poly[0] == poly[-1]:
                poly.pop()
            return poly

        # Inicia em cada interseção entrante não visitada na lista do polígono
        for node in subj_nodes:
            if node['inter'] and node['entry'] and not node['visited']:
                subpoly = follow_path(node)
                if len(subpoly) >= 3:
                    result.append(subpoly)

        return result
    
    def _calcular_intersecao_borda(self, v1, v2, borda):
        wxmin, wymin = self.window['wmin']
        wxmax, wymax = self.window['wmax']
        
        x1, y1 = v1
        x2, y2 = v2
        
        if borda == 'esquerda':
            if x2 - x1 != 0:
                t = (wxmin - x1) / (x2 - x1)
                return (wxmin, y1 + t * (y2 - y1))
        elif borda == 'direita':
            if x2 - x1 != 0:
                t = (wxmax - x1) / (x2 - x1)
                return (wxmax, y1 + t * (y2 - y1))
        elif borda == 'inferior':
            if y2 - y1 != 0:
                t = (wymin - y1) / (y2 - y1)
                return (x1 + t * (x2 - x1), wymin)
        elif borda == 'superior':
            if y2 - y1 != 0:
                t = (wymax - y1) / (y2 - y1)
                return (x1 + t * (x2 - x1), wymax)
        return None
    
    # Realiza clipping de todos os objetos no PPC
    def realizar_clipping(self):
        wxmin, wymin = self.window['wmin']
        wxmax, wymax = self.window['wmax']
        
        algoritmo = self.algoritmo_linha.get()
        
        for obj in self.objects:
            obj['coords_ppc_clipped'] = []
            obj['visivel'] = False
            
            if obj['type'] == 'ponto':
                # Clipping de ponto
                if len(obj['coords_ppc']) > 0:
                    x, y = obj['coords_ppc'][0]
                    if wxmin <= x <= wxmax and wymin <= y <= wymax:
                        obj['visivel'] = True
                        obj['coords_ppc_clipped'] = [(x, y)]
                    
            elif obj['type'] == 'reta':
                # Clipping de reta
                if len(obj['coords_ppc']) >= 2:
                    x1, y1 = obj['coords_ppc'][0]
                    x2, y2 = obj['coords_ppc'][1]
                    
                    if algoritmo == "Cohen-Sutherland":
                        aceito, p1, p2 = self.cohen_sutherland_clip(x1, y1, x2, y2)
                    else:  # Liang-Barsky
                        aceito, p1, p2 = self.liang_barsky_clip(x1, y1, x2, y2)
                    
                    if aceito:
                        obj['visivel'] = True
                        obj['coords_ppc_clipped'] = [p1, p2]
                    
            elif obj['type'] == 'poligono':
                # Clipping de polígono usando Weiler-Atherton
                if len(obj['coords_ppc']) >= 3:
                    poligonos_resultado = self.weiler_atherton_clip(obj['coords_ppc'])
                    if poligonos_resultado:
                        obj['visivel'] = True
                        # Armazena múltiplos polígonos se necessário
                        obj['coords_ppc_clipped'] = poligonos_resultado

    # Transforma um ponto do PPC para coordenadas de viewport
    def ppc_para_viewport(self, ponto):
        xppc, yppc = ponto
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        vpxmin, vpymin = self.viewport['vpmin']
        vpxmax, vpymax = self.viewport['vpmax']

        # Ajusta viewport para considerar margem de clipping
        viewport_width = vpxmax - vpxmin - 2 * self.clipping_margin
        viewport_height = vpymax - vpymin - 2 * self.clipping_margin
        
        window_width = wmaxx - wminx
        window_height = wmaxy - wminy

        if window_width == 0 or window_height == 0: 
            return (0, 0)
        
        # Normaliza em relação à window
        norm_x = (xppc - wminx) / window_width
        norm_y = (yppc - wminy) / window_height
        
        # Mapeia para viewport (invertendo Y)
        vp_x = self.clipping_margin + norm_x * viewport_width
        vp_y = (vpymax - vpymin) - (self.clipping_margin + norm_y * viewport_height)

        return (vp_x, vp_y)
    # Transforma todos os objetos PPC clipados para coordenadas de viewport
    def transformar_para_viewport(self):
        for obj in self.objects:
            obj['coords_viewport'] = []
            
            if not obj['visivel']:
                continue
            
            if obj['type'] == 'poligono':
                # Para polígonos, transforma cada subpolígono
                for poligono in obj['coords_ppc_clipped']:
                    pts_vp = [self.ppc_para_viewport(p) for p in poligono]
                    obj['coords_viewport'].append(pts_vp)
            else:
                # Para pontos e retas
                pts_vp = [self.ppc_para_viewport(p) for p in obj['coords_ppc_clipped']]
                obj['coords_viewport'] = pts_vp

    # Pipeline completo: Mundo -> PPC -> Clipping -> Viewport -> Desenho
    def _processar_e_desenhar(self):
        self.transformar_para_ppc()
        self.realizar_clipping()
        self.transformar_para_viewport()
        self.desenhar_viewport()
        self.desenhar_minimapa()
        self.atualizar_displaylist()

    def desenhar_viewport(self):
        self.canvas.delete("all")
        largura = int(self.canvas.cget('width'))
        altura = int(self.canvas.cget('height'))

        # Desenha a área externa região de clipping
        self.canvas.create_rectangle(0, 0, largura, altura, fill='lightgray', outline='')
        
        # Desenha a área interna região de renderização
        self.canvas.create_rectangle(
            self.clipping_margin, 
            self.clipping_margin, 
            largura - self.clipping_margin, 
            altura - self.clipping_margin, 
            fill='white', 
            outline='red',
            width=2
        )

        # Desenha objetos clipados
        for obj in self.objects:
            if not obj['visivel']:
                continue
                
            color = obj['color']
            
            if obj['type'] == 'ponto':
                if len(obj['coords_viewport']) > 0:
                    cx, cy = obj['coords_viewport'][0]
                    r = 3
                    self.canvas.create_oval(cx-r, cy-r, cx+r, cy+r, fill=color)
                
            elif obj['type'] == 'reta':
                if len(obj['coords_viewport']) >= 2:
                    p1_vp = obj['coords_viewport'][0]
                    p2_vp = obj['coords_viewport'][1]
                    self.canvas.create_line(p1_vp[0], p1_vp[1], p2_vp[0], p2_vp[1], width=2, fill=color)
                
            elif obj['type'] == 'poligono':
                # Desenha todos os polígonos resultantes do clipping
                for pts_vp in obj['coords_viewport']:
                    flat_pts = [coord for point in pts_vp for coord in point]
                    if len(flat_pts) >= 6:
                        self.canvas.create_line(*flat_pts, flat_pts[0], flat_pts[1], width=2, fill=color)

        # Mostra informações
        txt = (f"Algoritmo: {self.algoritmo_linha.get()}\n"
               f"Ângulo: {self.rotation_angle:.2f}°\n"
               f"Window Center: ({((self.window['wmin'][0] + self.window['wmax'][0]) / 2):.2f}, "
               f"{((self.window['wmin'][1] + self.window['wmax'][1]) / 2):.2f})")
            
        self.canvas.create_text(self.clipping_margin + 8, self.clipping_margin + 8, 
                               anchor='nw', text=txt, font=("Courier", 9), fill='blue')

    # desenhar a viewport do minimapa
    def desenhar_minimapa(self):
        self.minimap.delete("all")
        xs, ys = [], []
        for obj in self.objects:
            for x,y in obj['coords_mundo']:
                xs.append(x)
                ys.append(y)
        xs.extend([self.window['wmin'][0], self.window['wmax'][0], 
                   self.world_bounds['xmin'], self.world_bounds['xmax']])
        ys.extend([self.window['wmin'][1], self.window['wmax'][1], 
                   self.world_bounds['ymin'], self.world_bounds['ymax']])
        if not xs: 
            return
        xmin, xmax = min(xs), max(xs)
        ymin, ymax = min(ys), max(ys)
        if xmin == xmax: 
            xmin -= 1
            xmax += 1
        if ymin == ymax: 
            ymin -= 1
            ymax += 1
        pad_x = (xmax - xmin) * 0.08
        pad_y = (ymax - ymin) * 0.08
        xmin -= pad_x
        xmax += pad_x
        ymin -= pad_y
        ymax += pad_y
        mw = int(self.minimap.cget('width'))
        mh = int(self.minimap.cget('height'))

        def world_to_minimap(px, py):
            range_x = xmax - xmin
            range_y = ymax - ymin
            if range_x == 0 or range_y == 0:
                return 0, 0
            sx = mw / range_x
            sy = mh / range_y
            mx = (px - xmin) * sx
            my = mh - (py - ymin) * sy # inverte y para o minimapa
            return mx, my

        # Desenha limites do mundo
        p1 = world_to_minimap(self.world_bounds['xmin'], self.world_bounds['ymin'])
        p2 = world_to_minimap(self.world_bounds['xmin'], self.world_bounds['ymax'])
        p3 = world_to_minimap(self.world_bounds['xmax'], self.world_bounds['ymax'])
        p4 = world_to_minimap(self.world_bounds['xmax'], self.world_bounds['ymin'])
        self.minimap.create_polygon(p1[0],p1[1], p2[0],p2[1], p3[0],p3[1], p4[0],p4[1], 
                                    outline='gray', fill='lightgray', width=1)
        
        # Desenha objetos
        for obj in self.objects:
            color = obj['color']
            if obj['type'] == 'ponto':
                mx,my = world_to_minimap(*obj['coords_mundo'][0])
                self.minimap.create_oval(mx-2, my-2, mx+2, my+2, fill=color)
            else: # Trata 'reta' e 'poligono'
                pts = [world_to_minimap(*p) for p in obj['coords_mundo']]
                flat_pts = [c for p in pts for c in p]
                if len(flat_pts) >= 4:
                    if obj['type'] == 'poligono':
                        self.minimap.create_line(*flat_pts, flat_pts[0], flat_pts[1], width=1, fill=color)
                    else:
                        self.minimap.create_line(*flat_pts, width=1, fill=color)

        # Desenha a window rotacionada
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        cx = (wminx + wmaxx) / 2
        cy = (wminy + wmaxy) / 2
        w = wmaxx - wminx
        h = wmaxy - wminy
        
        angle_rad = math.radians(self.rotation_angle)
        cos_a = math.cos(angle_rad)
        sin_a = math.sin(angle_rad)

        corners = [(-w/2, -h/2), (w/2, -h/2), (w/2, h/2), (-w/2, h/2)]
        rotated_corners_world = []
        for x, y in corners:
            rx = x * cos_a - y * sin_a
            ry = x * sin_a + y * cos_a
            rotated_corners_world.append((cx + rx, cy + ry))
        
        minimap_corners = [world_to_minimap(*p) for p in rotated_corners_world]
        self.minimap.create_polygon(minimap_corners, outline='red', fill='', width=2)
    
    def _mover_e_recarregar(self, dx, dy):
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']

        window_width = wmaxx - wminx
        window_height = wmaxy - wminy

        world_width = self.world_bounds['xmax'] - self.world_bounds['xmin']
        world_height = self.world_bounds['ymax'] - self.world_bounds['ymin']

        if window_width >= world_width or window_height >= world_height:
            center_x = (self.world_bounds['xmin'] + self.world_bounds['xmax']) / 2
            center_y = (self.world_bounds['ymin'] + self.world_bounds['ymax']) / 2
            
            new_wminx = center_x - window_width / 2
            new_wminy = center_y - window_height / 2
            new_wmaxx = center_x + window_width / 2
            new_wmaxy = center_y + window_height / 2
            
            self.window['wmin'] = (new_wminx, new_wminy)
            self.window['wmax'] = (new_wmaxx, new_wmaxy)
        else:
            # Movimento em passos
            movimento = 2.0
            local_step_x = movimento * dx
            local_step_y = movimento * dy

            angle_rad = math.radians(self.rotation_angle)
            cos_a = math.cos(angle_rad)
            sin_a = math.sin(angle_rad)
            
            step_x = local_step_x * cos_a - local_step_y * sin_a
            step_y = local_step_x * sin_a + local_step_y * cos_a

            if wmaxx + step_x > self.world_bounds['xmax']:
                step_x = self.world_bounds['xmax'] - wmaxx
            if wminx + step_x < self.world_bounds['xmin']:
                step_x = self.world_bounds['xmin'] - wminx

            if wmaxy + step_y > self.world_bounds['ymax']:
                step_y = self.world_bounds['ymax'] - wmaxy
            if wminy + step_y < self.world_bounds['ymin']:
                step_y = self.world_bounds['ymin'] - wminy
        
            new_wminx = wminx + step_x
            new_wminy = wminy + step_y
            new_wmaxx = new_wminx + window_width
            new_wmaxy = new_wminy + window_height
        
            self.window['wmin'] = (new_wminx, new_wminy)
            self.window['wmax'] = (new_wmaxx, new_wmaxy)

        self._processar_e_desenhar()

    # Zoom in/out
    def _zoom(self, fator):
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
        
        width = wmaxx - wminx
        height = wmaxy - wminy
        
        new_width = width * fator
        new_height = height * fator
        
        world_width = self.world_bounds['xmax'] - self.world_bounds['xmin']
        world_height = self.world_bounds['ymax'] - self.world_bounds['ymin']
        if new_width > world_width:
            new_width = world_width
        if new_height > world_height:
            new_height = world_height
        MIN_SIZE = 1e-1
        if new_width < MIN_SIZE or new_height < MIN_SIZE:
            return
        
        center_x = (wminx + wmaxx) / 2
        center_y = (wminy + wmaxy) / 2
        
        new_wminx = center_x - new_width / 2
        new_wminy = center_y - new_height / 2
        new_wmaxx = center_x + new_width / 2
        new_wmaxy = center_y + new_height / 2
        
        if new_wmaxx > self.world_bounds['xmax']:
            deslocamento = self.world_bounds['xmax'] - new_wmaxx
            new_wmaxx += deslocamento
            new_wminx += deslocamento
        elif new_wminx < self.world_bounds['xmin']:
            deslocamento = self.world_bounds['xmin'] - new_wminx
            new_wminx += deslocamento
            new_wmaxx += deslocamento
        if new_wmaxy > self.world_bounds['ymax']:
            deslocamento = self.world_bounds['ymax'] - new_wmaxy
            new_wmaxy += deslocamento
            new_wminy += deslocamento
        elif new_wminy < self.world_bounds['ymin']:
            deslocamento = self.world_bounds['ymin'] - new_wminy
            new_wminy += deslocamento
            new_wmaxy += deslocamento
            
        self.window['wmin'] = (new_wminx, new_wminy)
        self.window['wmax'] = (new_wmaxx, new_wmaxy)
        
        self._processar_e_desenhar()

    # abrir janela da DisplayList
    def abrir_display_list(self):
        """Abre uma janela mostrando a DisplayList com todas as coordenadas"""
        # Se já existe uma janela, apenas traz para frente
        if self.displaylist_window is not None and self.displaylist_window.winfo_exists():
            self.displaylist_window.lift()
            return
            
        self.displaylist_window = tk.Toplevel(self.root)
        self.displaylist_window.title("DisplayList")
        self.displaylist_window.geometry("1100x450")
        
        # Callback para limpar referência quando fechar
        def on_closing():
            window = self.displaylist_window
            self.displaylist_window = None
            self.displaylist_tree = None
            if window:
                window.destroy()
        
        self.displaylist_window.protocol("WM_DELETE_WINDOW", on_closing)
        
        # Frame com scrollbar
        frame = tk.Frame(self.displaylist_window)
        frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        # Treeview
        cols = ("ID", "Tipo", "Cor", "Visível", "Coord.Mundo", "Coord.PPC", "Coord.Viewport")
        self.displaylist_tree = ttk.Treeview(frame, columns=cols, show="headings", height=15)
        
        # Configura colunas
        self.displaylist_tree.heading("ID", text="ID")
        self.displaylist_tree.heading("Tipo", text="Tipo")
        self.displaylist_tree.heading("Cor", text="Cor")
        self.displaylist_tree.heading("Visível", text="Visível")
        self.displaylist_tree.heading("Coord.Mundo", text="Coord. Mundo")
        self.displaylist_tree.heading("Coord.PPC", text="Coord. PPC")
        self.displaylist_tree.heading("Coord.Viewport", text="Coord. Viewport")
        
        self.displaylist_tree.column("ID", width=40)
        self.displaylist_tree.column("Tipo", width=80)
        self.displaylist_tree.column("Cor", width=60)
        self.displaylist_tree.column("Visível", width=60)
        self.displaylist_tree.column("Coord.Mundo", width=220)
        self.displaylist_tree.column("Coord.PPC", width=220)
        self.displaylist_tree.column("Coord.Viewport", width=220)
        
        # Scrollbar
        scrollbar = ttk.Scrollbar(frame, orient="vertical", command=self.displaylist_tree.yview)
        self.displaylist_tree.configure(yscrollcommand=scrollbar.set)
        
        self.displaylist_tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Preenche inicialmente
        self.atualizar_displaylist()

    # Atualiza a DisplayList
    def atualizar_displaylist(self):
        if self.displaylist_tree is None or not self.displaylist_window or not self.displaylist_window.winfo_exists():
            return
        
        # Limpa todos os itens
        for item in self.displaylist_tree.get_children():
            self.displaylist_tree.delete(item)
        
        # Preenche com dados atualizados
        idx = 1
        for obj in self.objects:
            tipo = obj['type']
            cor = obj['color']
            visivel = "Sim" if obj.get('visivel') else "Não"
            
            # Formata coordenadas
            mundo = self._formatar_coords(obj.get('coords_mundo', []))
            ppc = self._formatar_coords(obj.get('coords_ppc', []))
            
            # Viewport pode ter múltiplos polígonos
            if tipo == 'poligono' and obj.get('visivel'):
                vp = []
                for poly in obj.get('coords_viewport', []):
                    vp.append(self._formatar_coords(poly))
                viewport = "; ".join(vp) if vp else "[]"
            else:
                viewport = self._formatar_coords(obj.get('coords_viewport', []))
            
            # Adiciona tag para colorir linha se não visível
            tags = ("invisivel",) if not obj.get('visivel') else ()
            
            self.displaylist_tree.insert("", "end", 
                                        values=(idx, tipo, cor, visivel, mundo, ppc, viewport),
                                        tags=tags)
            idx += 1
        
        # Configura estilo para objetos invisíveis
        self.displaylist_tree.tag_configure("invisivel", foreground="gray")
    
    # Formata lista de coordenadas para exibição na DisplayList
    def _formatar_coords(self, coords):
        if not coords:
            return "[]"
        formatted = []
        for x, y in coords[:3]:
            formatted.append(f"({x:.2f},{y:.2f})")
        result = ", ".join(formatted)
        if len(coords) > 3:
            result += "..."
        return f"[{result}]"

    def _indent_xml(self, elem, level=0):
        i = "\n" + level * "\t"
        if len(elem):
            if not elem.text or not elem.text.strip():
                elem.text = i + "\t"
            if not elem.tail or not elem.tail.strip():
                elem.tail = i
            for child in elem:
                self._indent_xml(child, level + 1)
            if not child.tail or not child.tail.strip():
                child.tail = i
        else:
            if level and (not elem.tail or not elem.tail.strip()):
                elem.tail = i

    # gerar XML de saída com dados originais e transformados
    def gerar_xml_saida(self):
        if not self.caminho_arquivo:
            messagebox.showwarning("Aviso", "Nenhum arquivo foi carregado ainda!")
            return
        
        try:
            caminho_saida = filedialog.asksaveasfilename(
                defaultextension=".xml",
                filetypes=[("XML files", "*.xml"), ("All files", "*.*")],
                title="Salvar XML de Saída"
            )
            if not caminho_saida:
                return
            tree = ET.parse(self.caminho_arquivo)
            root = tree.getroot()
            
            comentario = ET.Comment(" Objetos após transformação PPC e clipping, convertidos para viewport ")
            root.append(comentario)
            
            for obj in self.objects:
                if not obj['visivel']:
                    continue
                
                if obj['type'] == 'ponto':
                    if len(obj['coords_viewport']) > 0:
                        x, y = obj['coords_viewport'][0]
                        ponto_elem = ET.SubElement(root, 'ponto')
                        ponto_elem.set('x', f"{x:.6f}")
                        ponto_elem.set('y', f"{y:.6f}")
                        ponto_elem.set('coords', 'viewport')
                        ponto_elem.set('cor', obj['color'])
                    
                elif obj['type'] == 'reta':
                    reta_elem = ET.SubElement(root, 'reta')
                    reta_elem.set('coords', 'viewport')
                    reta_elem.set('cor', obj['color'])
                    for x, y in obj['coords_viewport']:
                        ponto_elem = ET.SubElement(reta_elem, 'ponto')
                        ponto_elem.set('x', f"{x:.6f}")
                        ponto_elem.set('y', f"{y:.6f}")
                        
                elif obj['type'] == 'poligono':
                    # Pode ter múltiplos polígonos resultantes
                    for idx, pts_vp in enumerate(obj['coords_viewport']):
                        poligono_elem = ET.SubElement(root, 'poligono')
                        poligono_elem.set('coords', 'viewport')
                        poligono_elem.set('cor', obj['color'])
                        if len(obj['coords_viewport']) > 1:
                            poligono_elem.set('parte', str(idx + 1))
                        for x, y in pts_vp:
                            ponto_elem = ET.SubElement(poligono_elem, 'ponto')
                            ponto_elem.set('x', f"{x:.6f}")
                            ponto_elem.set('y', f"{y:.6f}")
                        
            self._indent_xml(root)
            
            tree.write(caminho_saida, encoding='utf-8', xml_declaration=True)
            messagebox.showinfo("Sucesso", f"XML de saída gerado com sucesso!\nSalvo em: {caminho_saida}")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao gerar XML de saída:\n{e}")

    def mostrar_controles_do_teclado(self):
        texto_controle = """
        Controles do Teclado ---------------------------------
        
        Movimentação da Window:
        - Teclas ← → ↑ ↓ : Move na direção correspondente.
        
        Zoom (10% por click):
        - Tecla '+' ou '=' : Aproxima (Zoom In).
        - Tecla '-' : Afasta (Zoom Out).
        
        Rotação da Window:
        - Tecla '>' ou '.' : Rotaciona para a direita.
        - Tecla '<' ou ',' : Rotaciona para a esquerda.
        
        Algoritmos Implementados:
        - Linhas: Cohen-Sutherland ou Liang-Barsky (selecionável)
        - Polígonos: Weiler-Atherton
        
        Pipeline de Transformação:
        Mundo → PPC → Clipping → Viewport → Desenho
        
        DisplayList:
        - Abre janela com visualização em tempo real do pipeline
        - Atualiza automaticamente a cada transformação
        """
        messagebox.showinfo("Controles do Teclado", texto_controle)


if __name__ == "__main__":
    root = tk.Tk()
    app = Visualizador(root)
    root.mainloop()
